## Compute driving distances
Here we modify our previous analysis that computed the Euclidean distance between candidate site (exits) and existing infrastructure (DCFC) by instead measuring *driving* distances. To do this we need a network dataset, often called a *graph dataset* of roads, which we get from OpenStreetMap, and a means for analyzing network datasets, which we can do via the `osmnx` package. 

The analysis here is guided by the analysis presented in [Lesson 6: Network Analysis in Python](https://automating-gis-processes.github.io/site/notebooks/L6/network-analysis.html) of the Automating GIS curriculum, but with a number of twists. We also need to dig deeper into the graph analysis capabilities of the [NetworkX package](https://networkx.github.io/documentation/networkx-1.10/index.html), specifically using its [shortest paths](https://networkx.github.io/documentation/networkx-1.10/reference/algorithms.shortest_paths.html) algorithms to measure network distances away from DFCF chargers. 

*Prior to running this notebook, you should have already run `A4-Fetch-NC-roads-as-graph.ipynb` to download the NC roads as a graphml dataset into your Data/OSM folder.*

The workflow here is:
* **Data import**:
 * Import major roads as a graph dataset from OSM saved graphml file
 * Import the DCFC csv file and convert to a geopandas dataframe
 * Import the exits feature class as a geopdandas dataframe
* **Network analysis**:
 * Identify the network nodes associated with each DCFC and Exit feature; add to their respective dataframes
 * Compute weighted distance from each node to all other nodes capped at 100 miles
 * Iterate through each node in the above result for DCFC nodes
 * For each DCFC source node, find destination nodes associated with Exits
 * Compare the distance associated with the DCFC-Exit node pair to any exiting Exit distance
 * If the Exit node had no distance associated with it, or if the distance associated with it is greater than the existing associated distance, then associate that distance with the Exit node (ensuring that the node is assigned the lowest distance value). 
* **Synthesis**:
 * Append node distances to exit features in exit geodataframe
 * Save the geodataframe to a shapefile

The result will be a geodataframe with a new column indicating driving distances to the nearest DCFC, up to 100 miles, which is saved to a new shapefile: `exits_drivedist.shp`

In [ ]:
#Import packages
import osmnx as ox
import networkx as nx
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point

### Load data into the coding environment

In [ ]:
#Load in the NC road network
nc_graph = ox.load_graphml('NC_roads.graphml',folder='../Data/OSM/')

In [ ]:
#Load in the DCFC locations as pandas dataframe
dcfc_df = pd.read_csv('../Data/NREL/DCFC.csv')

In [ ]:
#Convert to a geopandas dataframe
geom_points = [Point(xy) for xy in zip(dcfc_df['longitude'],dcfc_df['latitude'])]
dcfc_gdf = gpd.GeoDataFrame(dcfc_df,geometry=geom_points,crs=4326)

In [ ]:
#Load exits as a geopandas dataframe
exits_gdf = gpd.read_file('../Data/MJBA/Exits.shp')

### Compute network node ids for each DCFC and Exit locations
Use the `osmnx` package's [`get_nearest_node()`](https://osmnx.readthedocs.io/en/stable/osmnx.html#osmnx.geo_utils.get_nearest_node) function to identify the node closest to a given point, returning the ID of the node. Here we define a function that returns the ID of the closest node and then apply that function to the values in the geodataframe's *geometry* column, storing the node ID in a new column called `node_id`. We run this for both the DCFC and the Exits geodataframes, thereby allowing us to easily find the node in the NC roads network associated with the feature. 

In [ ]:
#Define a function to extract the nearest node ID for a point object
def get_nodeid(thePoint):
    #Get the yx tuple
    yx = (thePoint.y,thePoint.x)
    #Fetch the node nearest the xy tuple
    node_id = ox.get_nearest_node(nc_graph,yx)
    #Return the node id
    return node_id

In [ ]:
#Apply the function to each point in the DCFC geodataframe, adding node ID as a column
dcfc_gdf['node_id'] = dcfc_gdf['geometry'].apply(get_nodeid)

In [ ]:
#Apply the function to each point in the Exits geodataframe, adding node ID as a column
exits_gdf['node_id'] = exits_gdf['geometry'].apply(get_nodeid)

In [ ]:
#Create lists from the values, for later use
nodes_dcfc = dcfc_gdf['node_id'].unique()
nodes_exits = exits_gdf['node_id'].unique()

### Compute the distance away from each DCFC
With our network dataset, we can use the `networkx` package to compute distances along the network. This is done using the [Shortest Paths algorithms](https://networkx.github.io/documentation/networkx-1.10/reference/algorithms.shortest_paths.html), specifically the Dijkstra algorithms which computes a weighted distance (using lengths as our weight).  

---
##### Proof of concept
We'll explore this analysis using the example of a single DCFC location, computing the distance of each node away from this point, and then assigning this distance to each Exit feature by determining the node that is nearest the Exit. 

First, we'll examine how the shortest path algorithm is used: first to identify the paths within a set distance of a source, and then to compute the distances along these paths. 

Our examination will include the following:
* Extract a single node ID (from the list generated above)
* Explore how driving paths are extracted from a network dataset, supplying the source, the weight attribute, and the distance cutoff to the `single_source_dijkstra_path()` function. 
* Explore how driving distances are extracted from a network datasets, supplying the same inputs as above, but using the  `single_source_dijkstra_path_length()` algorithm.
* From this result, extract the node nearest each exit and assign the distance to that node

In [ ]:
#Extract an example start node: fetching the first one in the nodes_dcfc list
theStartNode = nodes_dcfc[0]
theStartNode

First, we'll extract the set of nodes falling within 100 miles (driving distance) of the source node

In [ ]:
#Extract the path (set of nodes) within 100 miles of the source node
thePaths = nx.single_source_dijkstra_path(G=nc_graph,
                                          source=theStartNode,
                                          weight='length',
                                          cutoff=100 * 1609.34)
type(thePaths)

This function returns a dictionary listing each terminal node within the cutoff range, and the nodes involved in getting to this terminal node. For example, we can reveal the nodes involved in getting to the node `169634378`.

In [ ]:
#Reveal the nodes involved in getting from the source to node 169634378
thePaths[169634378]

In [ ]:
#From that we can create a subgraph
subgraph = nx.subgraph(nc_graph,thePaths[169634378])

In [ ]:
#And we can plot that subgraph
ox.plot_graph_folium(subgraph)

That's just one path. To show all the paths within 100 miles, we can extract all the terminal nodes and use those to define our subgraph. (Note: this can be done more directly using the `networkx` package's `ego_graph()` function.)

In [ ]:
#Extract all paths and plot (this takes a moment...)
subgraph = nx.subgraph(nc_graph,thePaths.keys())
ox.plot_graph_folium(subgraph)

The above analysis reveals the *paths*, but what we really want are the *distances* between source (DCFC) and destination (Exit). So, we need to use the `single_source_dijkstra_path_length` function...

In [ ]:
#Compute distance to other nodes, with cutoff of 100 miles
theDistances = nx.single_source_dijkstra_path_length(G = nc_graph,
                                                     source = theStartNode,
                                                     cutoff = 100*1609.34,
                                                     weight = 'length')
type(theDistances)

The result of the algorithm is also a dictionary, this time listing nodes:distance pairs. For example we can see that the distance from the source DCFC to the node `169634378` is 638.83 meters:

In [ ]:
theDistances[169634378]

To explore some more we'll convert this dictionary to a dataframe for easier manipulation (e.g. to convert distances from meter to miles), and to enable visualization of the data. 

In [ ]:
#Create a dataframe of the distances, and plot in miles
df = pd.DataFrame()                     #Create an empty dataframe
df['node_id']=theDistances.keys()       #Add the node_ids to the dataframe
df['distance'] = theDistances.values()  #Add the distances (in meters) to the dataframe
df['distance'] = df['distance']/1609.34 #Convert distances to miles
df.head()

In [ ]:
#Plot a histogram of distances
df['distance'].hist();

---
##### Back to our analysis...
The above "proof of concept" examines network paths stemming from a single source node. We could iterate through each DCFC and run this process, but we have another `networkx` function that computes distances between **all** pairs of nodes in our network dataset, and from that result, we can pull out the paths originating from our DCFC source nodes. 

This is done by using the `all_pairs_dijkstra_path_length()` algorithm ([link](https://networkx.github.io/documentation/networkx-1.10/reference/generated/networkx.algorithms.shortest_paths.weighted.single_source_dijkstra_path.html#single-source-dijkstra-path)). The code for this is below, again applying a cutoff of 100 miles. 

In [ ]:
#Compute all pairs lengths
allDistPairs = nx.all_pairs_dijkstra_path_length(nc_graph,cutoff=100*1609.34,weight='length')
type(allDistPairs)

Because this function returns a large number of results, it returns these results as a "**generator**" object, not a dictionary or list. The items in a generator object can only be accessed sequentially (to avoid having to store the entire object in the computer's memory), so we need to iterate through the objects to work with it. 

Below, we loop through all items in this generator. Each item in the generator is a tuple, with the first item in this tuple being the `ID` of the source node, and the second item is a dictionary containing each node falling within 100 miles and its distance from the source node. For example, the result of the first item would be:
```
(1859256320, {1859256320: 0, 149376304: 42.997, 149412495: 59.782000000000004})
```
Here, `1859256320` is the source node, corresponding to the DCFC location. And `{1859256320: 0, 149376304: 42.997, 149412495: 59.782000000000004}` reveals the connected nodes and their distances (in meters). 

So, in the code below, we iterate through each item in the generator, and at each iteration, we check to see if the start node corresponds to a DCFC location. If it does, then we investigate whether its connected nodes include any Exit nodes (accomplished using [Python set intersections](https://www.w3schools.com/python/ref_set_intersection.asp)). And if it does, then we see if the distance between these two nodes is the minimum recorded distance. If it is, then we label that in the Edges dataframe; if not, then we keep the exising value...

In [ ]:
#Loop through the results and extract the distances associated with DCFC nodes
distance_data = {}
for distItem in allDistPairs:
    #Get the items in the tuple
    startNode_id = distItem[0]
    path_data = distItem[1]
    #Check whether the start node occurs in the list of DCFC nodes
    if startNode_id in nodes_dcfc:
        #If so, get its end nodes as a Python set object...
        endNodes = set(path_data.keys())
        #Now intersect this set with the set of exit node IDs
        valid_endNodes = endNodes.intersection(set(nodes_exits))
        #See if the intersection has any members
        if len(valid_endNodes) > 0:
            #If so, loop through items
            for node in valid_endNodes:
                #Get the distance associated with the node
                theDistance = path_data[node]
                #See if node already in dict
                if node in distance_data.keys():
                    #If so, compare existing distance to new distance
                    if distance_data[node] >= theDistance:
                        #If it's greater, update with smaller
                        distance_data[node] = theDistance
                #And if the node has not yet been added, add it
                else:
                    distance_data[node] = theDistance

With the `distance_data` dictionary created above, we can construct a dataframe listing each Exit node ID and its distance to the nearest DCFC node (just as we did previously)

In [ ]:
#Convert to a dataframe
df_distance = pd.DataFrame()
df_distance['node_id'] = distance_data.keys()
df_distance['distance'] = distance_data.values()
df_distance['distance'] = df_distance['distance'] / 1609.34
df_distance.head()

And now we can join these distance values to our existing Exits geodataframe, using the common `node_id` attribute to join the rows. (See this [link](https://www.tutorialspoint.com/python_pandas/python_pandas_merging_joining.htm) for a nice explanation of merging dataframes on a common attribute...)

In [ ]:
#Join the data to the exits data
exits_gdf1 = pd.merge(left=exits_gdf,    #"Left" side dataframe of the merge
                      right=df_distance, #"Right" side dataframe of the merge
                      left_on='node_id', #Attribute on the left dataframe used to join the frames
                      right_on='node_id',#Attribute on the right dataframe used to join the frames
                      how='left')        #Type of join: "left" keeps all records in left dataframe  

In [ ]:
#Save the data to a shapefile
exits_gdf1.to_file('../Data/processed/exits_distance.shp')

### A bit of visualization...

In [ ]:
#Plot a histogram of distances
exits_gdf1['distance'].hist(bins=10);

In [ ]:
#Create a list of exit colors, for plotting
exit_colors = []
for d in exits_gdf1['distance']:
    if d < 50: exit_colors.append('grey')  #Show exits <50 miles from DCFC as grey
    else: exit_colors.append('red')        #Show all other exits as red

In [ ]:
#Plot exits and charger locations
ax = exits_gdf1.plot(markersize='distance', #Make further exits larger
                     figsize=(20,15),       #Size of figure
                     color=exit_colors,     #Use colors set above
                     alpha=0.3)             #Set to mostly transparent
ax.set_title("NC Candidate Locations (Red=Exits between 50 and 100mi from DCFC)")

#Add the DCFC locations as small blue crosses
dcfc_gdf.plot(color='blue',marker='+',ax=ax)